In [ ]:
# python libraries
import numpy as np
from IPython.display import HTML, display

# pydrake imports
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder,
                         LinearQuadraticRegulator, Parser, StartMeshcat,
                         PlanarSceneGraphVisualizer, Simulator, Linearize,
                         FindResourceOrThrow,
                         MeshcatVisualizerCpp, MeshcatVisualizerParams, 
                         RigidTransform, Role, RollPitchYaw, Simulator, 
                         JointSliders)

In [ ]:
#start visualizer
meshcat = StartMeshcat()

Meshcat is now available at https://f68aa2bd-d5e5-4beb-aa3e-489f3e8c3132.deepnoteproject.com


In [ ]:
#Monkey bar urdf skeleton
monkey_bar = """ 
    <link name="barNUMBER">
        <inertial>
            <origin xyz="REPLACE 0 0" />
            <mass value="1" />
        </inertial>

        <visual>
            <origin xyz="REPLACE 0 0" />
            <geometry>
                <box size=".2 .2 .2" />
            </geometry>
            <material>
                <color rgba="0 0 0 1" />
            </material>
        </visual>
    </link>
"""

monkey_bar_joint = """
    <joint name="bar_jointNUMBER" type="fixed">
        <parent link="world" />
        <child link="barNUMBER" />
        <origin xyz="0 0 0" />
    </joint>
"""

In [ ]:
#monkey links
monkey_grasp_link = """
    <link name="grasp_link">
        <inertial>
            <origin xyz="0 0 -.5" />
            <mass value=".2" />
        </inertial>

        <visual>
            <origin xyz="0 0 -.5" />
            <geometry>
                <sphere radius=".05" />
            </geometry>
            <material>
                <color rgba="1 0 1 1" />
            </material>
        </visual>

        <visual>
            <origin xyz="0 0 -.5" />
            <geometry>
                <cylinder radius=".02" length="1" />
            </geometry>
            <material>
                <color rgba="0 0 1 1" />
            </material>
        </visual>
    </link>
"""

monkey_swing_link = """
    <link name="swing_link">
        <inertial>
            <origin xyz="0 0 .5" />
            <mass value=".2" />
        </inertial>

        <visual>
            <origin xyz="0 0 .5" />
            <geometry>
                <sphere radius=".05" />
            </geometry>
            <material>
                <color rgba="1 1 0 1" />
            </material>
        </visual>

        <visual>
            <origin xyz="0 0 .5" />
            <geometry>
                <cylinder radius=".02" length="1" />
            </geometry>
            <material>
                <color rgba="1 0 0 1" />
            </material>
        </visual>
    </link>
"""

In [ ]:
#monkey body base
monkey_base = """
    <link name="base">
        <inertial>
            <origin xyz="0 0 0" />
            <mass value="1" />
        </inertial>

        <visual>
            <origin xyz="0 0 0" />
            <geometry>
                <box size="0.3 0.3 0.3" />
            </geometry>
            <material>
                <color rgba="0 1 0 1" />
            </material>
        </visual>
    </link>
"""

base_joint_x = """
    <link name="floating_base_x" />

    <joint name="base_x" type="prismatic">
        <parent link="world" />
        <child link="floating_base_x" />
        <axis xyz="1 0 0" />
    </joint>
"""

base_joint_z = """
    <link name="floating_base_z" />

    <joint name="base_z" type="prismatic">
        <parent link="floating_base_x" />
        <child link="base" />
        <axis xyz="0 0 1" />
    </joint>
"""

In [ ]:
#joints
base_grasp_joint = """
    <joint name="base_grasp_joint" type="continuous">
        <parent link="base" />
        <child link="grasp_link" />
        <axis xyz="0 -1 0" />
        <origin xyz="0 0 -1" />
    </joint>
"""
monkey_joints = """
    <link name="floating_base_x" />

    <joint name="base_x" type="prismatic">
        <parent link="world" />
        <child link="floating_base_x" />
        <axis xyz="1 0 0" />
    </joint>

    <link name="floating_base_z" />

    <joint name="base_z" type="prismatic">
        <parent link="floating_base_x" />
        <child link="floating_base_z" />
        <axis xyz="0 0 1" />
    </joint>

    <joint name="grasp_joint" type="continuous">
        <parent link="floating_base_z" />
        <child link="grasp_link" />
        <axis xyz="0 -1 0" />
        <origin xyz="0 0 0" />
    </joint>

    <joint name="base_grasp_joint" type="fixed">
        <parent link="grasp_link" />
        <child link="base" />
        <origin xyz="0 0 -1" />
    </joint>

    <joint name="base_swing_joint" type="continuous">
        <parent link="base" />
        <child link="swing_link" />
        <axis xyz="0 -1 0" />
        <origin xyz="0 0 0" />
    </joint>
"""

In [ ]:
#compass gait bot urdf
compass_gait_urdf = """
<?xml version="1.0" ?>

<robot name="compass_gait">

  <!-- ground -->
  <!-- just for visualization, it is welded to world -->

  <link name="ground">

    <visual>
      <origin xyz="0 0 -.25" rpy="0 0 0"/>
      <geometry>
        <box size="10 1 .5" />
      </geometry>
      <material name="green">
        <color rgba="0 1 0 1" />
      </material>
    </visual>

  </link>

  <joint name="ground_is_world" type="fixed">

    <parent link="world" />
    <child link="ground" />
    <!-- slope of the ramp here -->
    <origin xyz="0 0 0" rpy="0 .04 0"/>

  </joint>

  <!-- floating base for stance foot -->

  <link name="floating_base_x" />

  <joint name="floating_base_x" type="prismatic">

    <parent link="ground"/>
    <child link="floating_base_x"/>
    <axis xyz="1 0 0"/>

  </joint>

  <link name="floating_base_z" />

  <joint name="floating_base_z" type="prismatic">

    <parent link="floating_base_x"/>
    <child link="floating_base_z"/>
    <axis xyz="0 0 1"/>

  </joint>

  <!-- stance leg -->

  <link name="stance_leg">

    <inertial>
      <origin xyz="0 0 .5"/>
      <mass value=".1"/>
    </inertial>

    <visual>
      <origin xyz="0 0 .5"/>
      <geometry>
        <sphere radius=".05"/>
      </geometry>
      <material>
        <color rgba="1 0 0 1"/>
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 .5"/>
      <geometry>
        <cylinder length="1" radius=".01"/>
      </geometry>
      <material>
        <color rgba="1 0 0 1"/>
      </material>
    </visual>

  </link>

  <joint name="floating_base_y" type="continuous">

    <parent link="floating_base_z"/>
    <child link="stance_leg"/>
    <axis xyz="0 -1 0"/>

  </joint>

  <!-- body, welded to stance leg -->

  <link name="body">

    <inertial>
      <origin xyz="0 0 1"/>
      <mass value="1"/>
    </inertial>

    <visual>
      <origin xyz="0 0 0"/>
      <geometry>
        <sphere radius=".1"/>
      </geometry>
      <material>
        <color rgba="1 1 1 1"/>
      </material>
    </visual>

  </link>

  <joint name="body_on_stance_leg" type="fixed">

    <parent link="stance_leg" />
    <child link="body" />
    <origin xyz="0 0 1"/>

  </joint>

  <!-- swing leg -->

  <link name="swing_leg">

    <inertial>
      <origin xyz="0 0 -.5"/>
      <mass value=".1"/>
    </inertial>

    <visual>
      <origin xyz="0 0 -.5"/>
      <geometry>
        <sphere radius=".05"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1"/>
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 -.5"/>
      <geometry>
        <cylinder length="1" radius=".01"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1"/>
      </material>
    </visual>

  </link>

  <joint name="hip_pin" type="continuous">

    <parent link="body"/>
    <child link="swing_leg"/>
    <axis xyz="0 -1 0"/>

  </joint>

</robot>
"""

In [ ]:
def create_monkey_urdf(bar_params, link_params=None):
    #bar params is [0, b_0, b_c]
    
    #urdf header and ender
    urdf = """<?xml version="1.0"?>\n<robot name="BattleMonkey">BASE</robot>"""
    # cart base of the robot
    urdf = urdf.replace("BASE", monkey_base + "BAR")

    bar_params[1] = -bar_params[1]
    for i in range(3):
        bar_x = bar_params[i]
        urdf = urdf.replace("BAR", monkey_bar.replace("NUMBER", str(i)).replace("REPLACE", str(bar_x), 2) + "BAR")
    urdf = urdf.replace("BAR", monkey_grasp_link + "BAR")
    urdf = urdf.replace("BAR", monkey_swing_link + "JOINT")
    #urdf = urdf.replace("BAR", "JOINT")

    for i in range(3):
        urdf = urdf.replace("JOINT", monkey_bar_joint.replace("NUMBER", str(i), 2) + "JOINT")
    
    #urdf = urdf.replace("JOINT", base_joint_x + "JOINT")
    #urdf = urdf.replace("JOINT", base_joint_z)
    urdf = urdf.replace("JOINT", monkey_joints)

    return urdf

In [ ]:
#create urdf
bar_params = [0, 1, 1.5]
monkey_urdf = create_monkey_urdf(bar_params)
print(monkey_urdf)

<?xml version="1.0"?>
<robot name="BattleMonkey">
    <link name="base">
        <inertial>
            <origin xyz="0 0 0" />
            <mass value="1" />
        </inertial>

        <visual>
            <origin xyz="0 0 0" />
            <geometry>
                <box size="0.3 0.3 0.3" />
            </geometry>
            <material>
                <color rgba="0 1 0 1" />
            </material>
        </visual>
    </link>
 
    <link name="bar0">
        <inertial>
            <origin xyz="0 0 0" />
            <mass value="1" />
        </inertial>

        <visual>
            <origin xyz="0 0 0" />
            <geometry>
                <box size=".2 .2 .2" />
            </geometry>
            <material>
                <color rgba="0 0 0 1" />
            </material>
        </visual>
    </link>
 
    <link name="bar1">
        <inertial>
            <origin xyz="-1 0 0" />
            <mass value="1" />
        </inertial>

        <visual>
            <origin xyz=

In [ ]:
meshcat.Delete()
meshcat.DeleteAddedControls()
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

# Load the file into the plant/scene_graph.
parser = Parser(plant)
#for name, directory in package_paths.items():
#    parser.package_map().Add(name, directory)
parser.AddModelFromString(monkey_urdf, "urdf")
#parser.AddModelFromString(compass_gait_urdf, "urdf")

plant.Finalize()

# Add two visualizers, one to publish the "visual" geometry, and one to
# publish the "collision" geometry.
visual = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat,
    MeshcatVisualizerParams(role=Role.kPerception, prefix="visual"))
#collision = MeshcatVisualizerCpp.AddToBuilder(
#    builder, scene_graph, meshcat,
#    MeshcatVisualizerParams(role=Role.kProximity, prefix="collision"))
# Disable the collision geometry at the start; it can be enabled by the
# checkbox in the meshcat controls.
#meshcat.SetProperty("collision", "visible", False)

sliders = builder.AddSystem(JointSliders(meshcat, plant))

diagram = builder.Build()
sliders.Run(diagram)

KernelInterrupted: Execution interrupted by the Jupyter kernel.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=18990445-f6e7-4d8c-9e5b-73c061f7343e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>